### The Battle of Neighbourhoods: London vs. Birmingham

##### - JImmy Chuong

In [1]:
import pandas as pd
import numpy as np

### Section: London

#### Web Scraping with Pandas

In [56]:
url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
londonDF = pd.read_html(url)
print("There are " + str(len(londonDF)) + " tables.")

TypeError: read_html() got an unexpected keyword argument 'sep'

In [44]:
londonDF

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 526         Woolwich                          Greenwich          LONDON   
 527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 529          Yeading                         Hillingdon           HAYES   
 

There are a total of 5 tables gathered from the html. We are interested in the second table containing the name of each Location, Borough, Post Town, and Postcode.

In [45]:
londonDF = londonDF[1]
londonDF

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


We are only interested in neighbourhoods in the London Post Town. And also only the Location, London Borough, and Postcode district columns.

In [46]:
londonDF = londonDF[londonDF['Post town'] == 'LONDON']
londonDF = londonDF.iloc[:, [0, 1, 3]]
londonDF

,Location,London borough,Postcode district
0,Abbey Wood,"Bexley, Greenwich [7]",SE2
1,Acton,"Ealing, Hammersmith and Fulham[8]","W3, W4"
6,Aldgate,City[10],EC3
7,Aldwych,Westminster[10],WC2
9,Anerley,Bromley[11],SE20
...,...,...,...
520,Wood Green,Haringey,N22
521,Woodford,Redbridge,"IG8, E18"
525,Woodside Park,Barnet,N12
526,Woolwich,Greenwich,SE18


In [55]:
londonDF['Postcode district']

KeyError: 'Postcode district'

#### Obtaining Latitude and Longitude coordinates with Geocoder

In [24]:
#pip install geocoder
# !conda install -c conda-forge geopy --yes
# !conda install folium -c conda-forge
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests

In [30]:
geolocator = Nominatim(user_agent='london_explorer')
londonDF['Coordinates'] = (londonDF[0] +', ' + londonDF[1] + ', ' + londonDF[2]).apply(geolocator.geocode)
londonDF['Latitude'] = londonDF['Coordinates'].apply(lambda x: x.latitude)
londonDF['Longitude'] = londonDF['Coordinates'].apply(lambda x: x.longitude)

KeyError: 0

In [39]:
london = londonDF[:, 0]
london

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [40]:
londonDF

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
6,Aldgate,City[10],LONDON,EC3,020,TQ334813
7,Aldwych,Westminster[10],LONDON,WC2,020,TQ307810
9,Anerley,Bromley[11],LONDON,SE20,020,TQ345695
...,...,...,...,...,...,...
520,Wood Green,Haringey,LONDON,N22,020,TQ305905
521,Woodford,Redbridge,LONDON,"IG8, E18",020,TQ405915
525,Woodside Park,Barnet,LONDON,N12,020,TQ256925
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
